# Laser Workshop 5s Model APIs

## iiotuxpc

### Tasks

- check validity of client id and  client secret by generating bearer token once manually before demonstartion.
- get bearer token every 25 minutes `bearerToken()`.
- get asset id and aspect name only once in lifetime `get_assetid()`.
- API calls to write time series data `put_timeseries()`
  - 5s_Model
    - logic_validation
    - input_validation
    - station_status
    - queue_status
  - RCT_server
    - RCT_server

### Import packages

In [1]:
import requests
import json
import base64
import datetime

### Main functions

In [2]:
#=========================================================
#--- Generate bearer_token
#=========================================================

#--- client id and client secret is hard coded.
def bearerToken():
    #--- App details from developer cockpit
    CLIENT_ID = "iiotuxpc-testdm-v1.0.0"
    CLIENT_SECRET = "UvwnNgdc8Y0KsE7IZSFy2Jp28kQGzAd0fEHXcDOWbIX"
    APP_NAME = "testdm"
    APP_VERSION = "v1.0.0"
    HOST_TENANT = "iiotuxpc"
    USER_TENANT = "iiotuxpc"
    credential_create_time = "23/02/2023::18:53"

    #--- generate access token
    id = CLIENT_ID
    secret = CLIENT_SECRET
    encode_msg = id + ":" + secret
    token = base64.b64encode(encode_msg.encode("utf-8")).decode("utf-8")
    url = "https://gateway.eu1.mindsphere.io/api/technicaltokenmanager/v3/oauth/token"

    payload = json.dumps({
    "grant_type": "client_credentials",
    "appName": APP_NAME,
    "appVersion": APP_VERSION,
    "hostTenant": HOST_TENANT,
    "userTenant": USER_TENANT
    })
    headers = {
    'X-SPACE-AUTH-KEY': "Basic " + token,
    'Content-Type': 'application/json'
    }

    token_response = requests.request("POST", url, headers=headers, data=payload)
    # print(token_response.text)
    bearer_token = json.loads(token_response.text)["access_token"]
    return(bearer_token)
#------------------------------------------------------------------------------------------------------

#=========================================================
#--- Find AssetID using Asset name with GET API
#=========================================================

#--- Have to be executed only once in lifetime as the asset id do not change.
def get_assetid(asset_name,bearer_token = None):
    #--- get list of assets
    if bearer_token == None:
        bearer_token = bearerToken()
    url = "https://gateway.eu1.mindsphere.io/api/assetmanagement/v3/assets"
    payload={}
    headers = {
    'Authorization': 'Bearer ' + bearer_token
    }
    assets_list_response = requests.request("GET", url, headers=headers, data=payload)  # GET API call for assets_list
    assets_list=json.loads(assets_list_response.text)
    assets_count = assets_list["page"]["totalElements"]     # total number of assets available
    
    #--- find the asset_name and the corresponding ID in the assets_list
    for ii in range(0,assets_count):
        if assets_list["_embedded"]["assets"][ii]["name"] == asset_name:
            asset_Id = assets_list["_embedded"]["assets"][ii]["assetId"]
            # print(asset_name,asset_Id)
            break
    else: print("Error:\n\tAsset not found in the assets list.\n\tVerify the accuracy of asset name and the assets list. Asset name is case sensitive.")
    return(asset_Id)
#------------------------------------------------------------------------------------------------------

#=========================================================
#--- GET time series data
#=========================================================

# get time series data from MindSphere specific to the aspect of a asset and query condition
def get_timeseries(asset_Id, aspect_name, from_time, to_time,bearer_token = None):
    if bearer_token == None:
        bearer_token = bearerToken()
    
    data_condition = "?from=" + from_time + "&to=" + to_time    # query condition with from and to time period
    end_point = "iottimeseries/v3/timeseries/" + asset_Id + "/" + aspect_name + data_condition
    base_url = "https://gateway.eu1.mindsphere.io/api/"
    url = base_url + end_point

    payload={}
    headers = {
     'Authorization': 'Bearer ' + bearer_token
    }

    time_series_response = requests.request("GET", url, headers=headers, data=payload)  # GET API call
    print(time_series_response.text)
    time_series=json.loads(time_series_response.text)
    return(time_series)

#------------------------------------------------------------------------------------------------------

#=========================================================
#--- PUT time series data
#=========================================================

# put time series data into MindSphere specific to the aspect of a asset
def put_timeseries(asset_Id, aspect_name, payload,bearer_token = None):
    if bearer_token == None:
        bearer_token = bearerToken()
    
    end_point = "iottimeseries/v3/timeseries/" + asset_Id + "/" + aspect_name
    base_url = "https://gateway.eu1.mindsphere.io/api/"
    url = base_url + end_point

    headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'Authorization': "Bearer " + bearer_token
    }

    response = requests.request("PUT", url, headers=headers, data=payload)  # PUT API call
    if not response.text:
        print("Data PUT successfully.")
    else:print("Error:\n\t",response.text)

#------------------------------------------------------------------------------------------------------


### Demonstration - Initialization

In [32]:
#--- find asset_id to use in the API calls
#-- no need to run if already available
Id_5s_Model = get_assetid("5s_Model")
Id_RCT_server = get_assetid("RCT_server")

print(Id_5s_Model)
print(Id_RCT_server)

7da127f5a566408db16e0aeace5181e9
f018a3e7cc4343efa5cc9eced3cc11d5


### Demostration - API Handles

In [31]:
# bearer_token = bearerToken()    #-- Expires in 1799s. So, has to repeat this call every 25 to 29 minutes.

#--- writing logic_validation
indicator = [0.9,0.82]  #-- use this variable to pass the value #-- double
indicator_data = json.dumps([
    {
    "_time" : datetime.datetime.utcnow().isoformat() + 'Z',
    "logic" : indicator[0],
    "input" : indicator[1]
    }
])
put_timeseries(asset_Id=Id_5s_Model,aspect_name="validation_indicator",payload=indicator_data,bearer_token=bearer_token)

#--- writing station_status
station_status = [False,True,True,False,True]    #-- use this variable to pass the value    #-- Boolean    #-- [True,False] = [Busy,Idle]
station_data = json.dumps([
    {
    "_time" : datetime.datetime.utcnow().isoformat() + 'Z',
    "station_1" : station_status[0],
    "station_2" : station_status[1],
    "station_3" : station_status[2],
    "station_4" : station_status[3],
    "station_5" : station_status[4]
    }
])
put_timeseries(asset_Id=Id_5s_Model,aspect_name="station_status",payload=station_data,bearer_token=bearer_token)

#--- writing queue_status (number of parts)
queue_status = [1,3,2,9,1]    #-- use this variable to pass the value   #-- integer
queue_data = json.dumps([
    {
    "_time" : datetime.datetime.utcnow().isoformat() + 'Z',
    "queue_1" : queue_status[0],
    "queue_2" : queue_status[1],
    "queue_3" : queue_status[2],
    "queue_4" : queue_status[3],
    "queue_5" : queue_status[4]
    }
])
put_timeseries(asset_Id=Id_5s_Model,aspect_name="queue_status",payload=queue_data,bearer_token=bearer_token)

#--- writing RCT_server
RCT_data = [55, 23.55, 22.65,2]     #-- use this variable to pass the value   #-- [part_id, path_1, path_2, queue_id]  #-- [integer, double, double, integer]
RCT_data = json.dumps([
    {
    "_time" : datetime.datetime.utcnow().isoformat() + 'Z',
    "part_id" : RCT_data[0],
    "path_1" : RCT_data[1],
    "path_2" : RCT_data[2],
    "queue_id" : RCT_data[3]
    }
])
put_timeseries(asset_Id=Id_RCT_server,aspect_name="RCT_server",payload=RCT_data,bearer_token=bearer_token)

Data PUT successfully.
Data PUT successfully.
Data PUT successfully.
Data PUT successfully.
